In [ ]:
# Importação de bibliotecas
import json
import openai

In [ ]:
# Cria cleinte para os serviços da OpenAI
client = openai.Client()

In [6]:
# Função de saudação por Período (manhã, tarede, noite)
def saudacao_por_periodo(hora):
    if 5 <= hora < 12:
        return json.dumps({"saudacao":"Bom dia!"})
    elif 12 <= hora < 18:
        return json.dumps({"saudacao":"Boa tarde!"})
    elif 18 <= hora < 22:
        return json.dumps({"saudacao":"Boa noite!"})
    else:
        return json.dumps({"saudacao":"Boa madrugada!"})
    

In [ ]:
# Vamos criar TOOLS, que são ferramentas que a OpenAI oferece aos DEV's
tools = [{
    "type":"function",
    "function": {
        "name": "saudacao_por_periodo",
        "description": "Retorna uma saudação baseada na uma hora do dia",
        "parameters": {
            "type": "object",
            "properties": {
                "hora": {
                    "type": "integer",
                    "description": "A hora do dia em formato de 24h"},},
                "required": ["hora"]
        }}}]

In [ ]:
# Dicionário de mapeamento entre a TOOL e a FUNÇÃO
funcao_disponivel = {
    "saudacao_por_periodo": saudacao_por_periodo}

In [16]:
mensagens = [{"role": "user", "content": "Quala a saudação o modelo me dá se for 23h?"}]

In [17]:
resposta = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=mensagens,
    tools=tools,
    tool_choice="auto")

In [18]:
mensagem_resp = resposta.choices[0].message
mensagem_resp

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_WP3r2amf1pu5azNH3LRyA2XG', function=Function(arguments='{"hora":23}', name='saudacao_por_periodo'), type='function')])

In [19]:
tool_calls = mensagem_resp.tool_calls
tool_calls

[ChatCompletionMessageFunctionToolCall(id='call_WP3r2amf1pu5azNH3LRyA2XG', function=Function(arguments='{"hora":23}', name='saudacao_por_periodo'), type='function')]

In [ ]:
if tool_calls:
    mensagens.append(mensagem_resp)
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = funcao_disponivel[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            hora = function_args.get("hora")
        )

        mensagens.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response})
    
    segunda_resposta = client.chat.completions.create(
        model = "gpt-3.5-turbo-0125",
        messages = mensagens
    )

In [23]:
mensagem_resposta = segunda_resposta.choices[0].message
mensagem_resposta

ChatCompletionMessage(content='Boa madrugada!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)